In [23]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
def inner_prod(A,B):
    p_n, _ = A.shape
    return 1./p_n * np.trace(A.dot(B.T))

def norm(A):
    return inner_prod(A,A)

def estimator(X):
    S = np.cov(X.T)
    n, p = X.shape
    m = inner_prod(S, np.eye(p))
    d = norm(S - m * np.eye(p))

    bar_b = 0
    for x in X:
        xk = np.reshape(x, (p,1))
        bar_b += norm(xk.dot(xk.T) - S)
    bar_b = bar_b / n**2
    b = min(bar_b, d)
    a = d - b

    S_hat = b/d*m*np.eye(p) + a/d*S
    return S_hat

In [2]:
ret = pd.read_csv('data/ret.csv', header = None).values 
#ret.shape = (10344, 3251), (day, stock)
univ = pd.read_csv('data/topMV95.csv', header = None).values 
univ -= 1 #because Matlab is 1 indexed
#univ.shape = (360, 1000), (OOS month, sorted list of stocks to consider for that period)
dates = pd.read_csv('data/mydatestr.txt', header = None, parse_dates = [0]) 
#dates.shape = (10344, 1), date for each day in return (not a numpy array, but a dataframe with DT objects)
tradeidx = pd.read_csv('data/investDateIdx.csv', header = None).values 
tradeidx -= 1 #because Matlab is 1 indexed!
#tradeidx.shape = (360, 1), (row of univ -> index in ret matrix)
ret[ret == -500] = np.nan
ret = ret / 100 #ret is in percent
ret_nonan = ret.copy()
ret_nonan[np.isnan(ret)] = 0
#ret[np.isnan(ret)] = 0 #sometimes we're missing OOS returns
meta_info = [ret, ret_nonan, univ, tradeidx]

In [21]:
#Sanity check with Michael's code
today = tradeidx[0][0]
#print(today)
pastPeriod = range(today-1260,today)
#print(pastPeriod)
investPeriod = range(today, today+21)
#print(investPeriod)
universe = univ[0, :100]
#print(universe)
pastData = ret[pastPeriod][:,universe]
investData = ret[investPeriod][:,universe]
combinedData = np.concatenate((pastData, investData))
print(combinedData.min())
print(combinedData.max())
print(combinedData.mean())

-0.31855
0.22359
0.0008121786494925839


In [22]:
np.save("pastData", pastData)

In [17]:
np.linalg.inv(shrink_LIN(pastData))

array([[ 6.33790357e+03,  1.49449866e+02,  1.09006795e+02, ...,
         1.54034053e+02,  1.45088615e+02,  4.21312458e+01],
       [ 1.49449866e+02,  4.69349211e+03, -1.11100386e+02, ...,
         1.13626090e+02,  7.93712805e+00, -3.36640944e+01],
       [ 1.09006795e+02, -1.11100386e+02,  8.34575520e+03, ...,
        -2.23098926e+02,  1.07829845e+02, -6.41821853e+01],
       ...,
       [ 1.54034053e+02,  1.13626090e+02, -2.23098926e+02, ...,
         5.40271566e+03,  2.19869975e+02,  1.93401044e+02],
       [ 1.45088615e+02,  7.93712805e+00,  1.07829845e+02, ...,
         2.19869975e+02,  8.86281790e+03, -2.44812433e+02],
       [ 4.21312458e+01, -3.36640944e+01, -6.41821853e+01, ...,
         1.93401044e+02, -2.44812433e+02,  4.42129062e+03]])

In [11]:
def optimal_weights(cov):
    n = cov.shape[0]
    prec = np.linalg.inv(cov)
    denom = np.matmul(np.matmul(np.ones(n), prec), np.ones(n))
    return np.matmul(prec, np.ones(n)) / denom

In [12]:
def sample_cov_nonan(pastRet):
    df = pd.DataFrame(pastRet)
    clean_cov = df.cov() #does this by removing nans
    if np.count_nonzero(np.isnan(clean_cov)) > 0:
        print("{} nans in covariance matrix".format(np.count_nonzero(np.isnan(clean_cov))))
        assert False
    return clean_cov
    #return np.cov(pastRet.T) (this is with numpy)

In [13]:
def MTP2_cov(pastRet):
    return solver(pastRet)

In [14]:
def shrunk_sample_cov(lamb):
    return lambda pastRet: (1-lamb) * np.cov(pastRet.T) + np.eye(pastRet.shape[1]) * lamb

def equal_weight(pastRet):
    _, N = pastRet.shape
    return np.eye(N)

def shrink_LIN(pastRet):
    return estimator(pastRet)

In [10]:
def getCumRet_sanity_check(outret):
    cum_rets = []
    D, N = outret.shape #D is days, N is number of stocks
    for i in range(N):
        stock_ret = 1.
        for d in range(D):
            day_ret = outret[d, i]
            stock_ret *= (1+day_ret) #day_ret should NOT be in percent
        cum_rets.append(stock_ret - 1.) #to convert back to percentage increase
    return cum_rets #N for each stock

In [11]:
def OOS_rets(w, outret):
    assert np.isclose(np.sum(w), 1.0)
    total_ret = outret + 1
    cum_ret = np.cumprod(total_ret, axis=0)[-1,:]
    cum_ret -= 1
    return np.dot(w, cum_ret)

In [13]:
def get_OOS_rets(cov_estimator, meta_info):
    ret, ret_nonan, univ, tradeidx = meta_info
    rets = []
    opt_rets = []
    N = 200 #Number of stocks
    T = 1260 #lookback (units of days) --> should be in months though
    P = 1 #lookahead (units of months, since we multiply by 20)
    print('Lookback window in months', T // 20)
    print('Lookback window in years', (T // 20) // (252 / 20))
    
    start = time.time()
    for h in range(5):#len(univ)):
        universe = univ[h,:N]
        today = tradeidx[h][0]
        date = dates.iloc[today].values[0]
        pastPeriod_startdate = dates.iloc[today-T].values[0]
        #print("Past period start date", pastPeriod_startdate, "Period start date", date)

        pastPeriod = range(today-T, today)            
        pastRet = ret[pastPeriod][:, universe]
        
        #This is for back when we were worried about nans
        #nan_perc = np.isnan(pastRet).sum(axis = 0)  / T
        #nan_thres = nan_perc < 0.0025 #only include if less than 2.5% is nan
        #new_universe = universe[nan_thres]
        #print("Number in new universe: {} out of {}".format(new_universe.shape[0], N))
        #pastRet = ret_nonan[pastPeriod][:, new_universe]
        
        cov = cov_estimator(pastRet)
        w = optimal_weights(cov)
        
        #sns.heatmap(np.cov(pastRet.T))
        #S = np.cov(pastRet.T)
        #print('Less than 0: ', np.sum(S < 0.))
        
        #curret = OOS_rets(w, outRet)
        investPeriod = range(today, today + P*21)
        outRet = ret[investPeriod][:, universe]
        #outRet = ret_nonan[investPeriod][:, new_universe] #don't want to include nans
        curret = retConstShare(outRet, w)
        rets.append(curret)
        
#         if include_opt:
#             #print(sorted(np.linalg.eigvals(np.cov(outRet.T))))
#             opt_w = optimal_weights(np.cov(outRet.T))
#             opt_curret = retConstShare(outRet, opt_w)
#             opt_rets.append(opt_curret)

#     if include_opt:
#         return rets, opt_rets
        end = time.time()
        print(end - start)
        start = end
    return rets

In [14]:
def retConstShare(retMat, w):
    n, p = retMat.shape
    if len(w.shape) == 1:
        w = np.expand_dims(w, 1)
    assert(w.shape == (p,1))
    wSum1 = w/np.sum(w)

    totalRetMat = 1 + retMat

    cummProdd = np.cumprod(totalRetMat, axis = 0)
    navVec = np.matmul(cummProdd, wSum1)

    wEnd = cummProdd[n-1, :]
    wEnd = np.dot(wEnd, w) #since w is (p,1) but wEnd is (1,p)
    wEnd = wEnd/np.sum(wEnd)
    wEnd = wEnd.T

    navVecTot = np.concatenate((np.ones((1,1)), navVec[:(n-1),]))

    totalRetVec = np.divide(navVec, navVecTot)

    retVec = totalRetVec - 1
    retVec = retVec * np.sum(w)

    return np.sum(retVec) #sum of all of the returns

In [15]:
def print_normalize(a):
    print(np.mean(a) * 252 / 20, np.std(a) * np.sqrt(252 / 20))

In [16]:
equal_rets = get_OOS_rets(equal_weight, meta_info) #covariance matrix is identity

Lookback window in months 63
Lookback window in years 5.0
0.062419891357421875
0.06939911842346191
0.06674790382385254
0.0717620849609375
0.06377100944519043


In [17]:
print_normalize(equal_rets)

0.4392591113954154 0.07427618397835759


In [18]:
shrink_LIN_rets = get_OOS_rets(shrink_LIN, meta_info) #covariance matrix is identity

Lookback window in months 63
Lookback window in years 5.0
0.4039430618286133
0.3930699825286865
0.4225599765777588
0.399914026260376
0.4133110046386719


In [19]:
print_normalize(shrink_LIN_rets)

0.5610445293944472 0.06327852009388521


In [ ]:
MTP2_rets = get_OOS_rets(MTP2_cov, meta_info)

In [ ]:
print_normalize(MTP2_rets)

In [112]:
for a in [equal_rets, 
          shrink_LIN_rets,
         ]:
    print_normalize(a)

0.13307000453998713 0.18581436079362454
0.11095697546625338 0.11953342106182747
